# [Hate Speech Identification Shared Task](https://multihate.github.io/): Subtask 1B at [BLP Workshop](https://blp-workshop.github.io/) @IJCNLP-AACL 2025

This shared task is designed to identify the type of hate, its severity, and the targeted group from social media content. The goal is to develop robust systems that advance research in this area.

In this subtask, given a Bangla text collected from YouTube comments, categorize whether the hate towards individuals, organizations, communities, or society.

### Downloading dataset from github

In [1]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_dev_test.tsv

--2025-07-11 07:21:41--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8005567 (7.6M) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1B_train.tsv’

blp25_hatespeech_su 100%[===================>]   7.63M  --.-KB/s    in 0.08s   

2025-07-11 07:21:41 (91.6 MB/s) - ‘blp25_hatespeech_subtask_1B_train.tsv’ saved [8005567/8005567]

--2025-07-11 07:21:41--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.co

### installing required libraries.
 - transformers
 - datasets
 - evaluate
 - accelerate

In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate
# !pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00


#### importing required libraries and setting up logger

In [3]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

### Defining the training, validation, and test data

In [12]:
train_file = 'blp25_hatespeech_subtask_1B_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1B_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1B_dev_test.tsv'

### Disable wandb

In [29]:
import os
os.environ["WANDB_DISABLED"] = "true"

### Setting up the training parameters

In [31]:
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir="./distilBERT_m/",
    overwrite_output_dir=True,
    remove_unused_columns=False,
    local_rank= 1,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="no",
    report_to=None
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
max_seq_length = 512
batch_size = 16

[INFO|training_args.py:2186] 2025-07-11 07:25:11,523 >> PyTorch: setting up devices
[INFO|training_args.py:1863] 2025-07-11 07:25:11,542 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[WARNING|integration_utils.py:101] 2025-07-11 07:25:11,544 >> Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [5]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=Interv

#### Defining the Model

In [6]:
model_name = 'distilbert-base-multilingual-cased'

#### setting the random seed

In [8]:
set_seed(training_args.seed)

#### Loading data files

In [13]:
l2id = {'None': 0, 'Society': 1, 'Organization': 2, 'Community': 3, 'Individual': 4}
train_df = pd.read_csv(train_file, sep='\t')
# print(train_df['label'])
train_df['label'] = train_df['label'].map(l2id).fillna(0).astype(int)
train_df = Dataset.from_pandas(train_df)
validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['label'] = validation_df['label'].map(l2id).fillna(0).astype(int)
validation_df = Dataset.from_pandas(validation_df)
test_df = pd.read_csv(test_file, sep='\t')
#test_df['label'] = test_df['label'].map(l2id)
test_df = Dataset.from_pandas(test_df)

data_files = {"train": train_df, "validation": validation_df, "test": test_df}
for key in data_files.keys():
    logger.info(f"loading a local file for {key}")
raw_datasets = DatasetDict(
    {"train": train_df, "validation": validation_df, "test": test_df}
)

INFO:__main__:loading a local file for train
INFO:__main__:loading a local file for validation
INFO:__main__:loading a local file for test


In [14]:
len(test_df['id'])

2512

##### Extracting number of unique labels

In [15]:
# Labels
label_list = raw_datasets["train"].unique("label")
print(label_list)
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)

[0, 4, 1, 3, 2]


### Loading Pretrained Configuration, Tokenizer and Model

In [16]:
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    finetuning_task=None,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    from_tf=bool(".ckpt" in model_name),
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

[INFO|configuration_utils.py:711] 2025-07-11 07:23:48,079 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/config.json
[INFO|configuration_utils.py:774] 2025-07-11 07:23:48,137 >> Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weight

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

[INFO|configuration_utils.py:711] 2025-07-11 07:23:48,384 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/config.json
[INFO|configuration_utils.py:774] 2025-07-11 07:23:48,387 >> Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.53.1",
  "vocab_size": 119547
}



vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

[INFO|tokenization_utils_base.py:2012] 2025-07-11 07:23:49,623 >> loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/vocab.txt
[INFO|tokenization_utils_base.py:2012] 2025-07-11 07:23:49,624 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/tokenizer.json
[INFO|tokenization_utils_base.py:2012] 2025-07-11 07:23:49,625 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2012] 2025-07-11 07:23:49,625 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:2012] 2025-07-11 07:23:49,626 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/tokenizer_config.json
[INFO|tokenization_u

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

[INFO|modeling_utils.py:1267] 2025-07-11 07:24:06,221 >> loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/model.safetensors
[INFO|modeling_utils.py:5374] 2025-07-11 07:24:06,465 >> Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing

#### Preprocessing the raw_datasets

In [17]:
non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
sentence1_key= non_label_column_names[1]

# Padding strategy
padding = "max_length"

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",)

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

if 128 > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({128}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}.")
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)


Running tokenizer on dataset:   0%|          | 0/35522 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

#### Finalize the training data for training the model

In [18]:
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))

In [19]:
train_dataset

Dataset({
    features: ['id', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 35522
})

#### Finalize the development/evaluation data for evaluating the model

In [20]:
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))

#### Finalize the test data for predicting the unseen test data using the model

In [21]:
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))

#### Log a few random samples from the training set

In [22]:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

INFO:__main__:Sample 7296 of the training set: {'id': 660, 'text': 'সরকারের দায়িত্বপ্রাপ্ত সংস্থা সকল বাণিজ্যিক ভবন গুলোকে বছরে অন্তত একবার পরিদর্শন করে রিপোর্ট প্রদান করবেন একটা বাণিজ্যিক ভবনে নিরাপত্তার সকল ব্যবস্থা থাকা উচিত কিন্তু এই সব সম্ভবের দেশে কিছুই করা হয় না এই দেশে মানুষের জীবনের কোন দাম নেই তাই কেউ কোনকিছুর তদারকিরও দরকার মনে করে না', 'label': 0, 'input_ids': [101, 62853, 11421, 100, 978, 22756, 102069, 87664, 17660, 20513, 80198, 15215, 15691, 971, 75762, 950, 30277, 16431, 18243, 32465, 11199, 937, 70115, 13542, 28777, 36213, 29740, 12235, 17511, 11128, 111240, 40433, 13334, 974, 12235, 22335, 47719, 33072, 76677, 948, 11128, 49178, 11737, 28777, 36715, 17660, 20513, 80198, 15215, 15691, 971, 75762, 11199, 967, 27268, 96397, 13542, 72659, 11128, 87664, 970, 15215, 106676, 964, 81661, 66449, 17460, 35003, 14112, 100024, 978, 15002, 111240, 61596, 100047, 88324, 11199, 37855, 14998, 14704, 100, 26109, 14112, 88324, 11199, 109216, 955, 13100, 75762, 11421, 41431, 965, 58354

#### Get the metric function `accuracy`

In [23]:
metric = evaluate.load("accuracy")

#### Predictions and label_ids field and has to return a dictionary string to float.

In [24]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


#### Data Collator

In [25]:
data_collator = default_data_collator

#### Initialize our Trainer

In [26]:
train_dataset = train_dataset.remove_columns("id")
eval_dataset = eval_dataset.remove_columns("id")

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-32-1049306949.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


#### Training our model

In [33]:
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))



[INFO|trainer.py:2402] 2025-07-11 07:25:21,245 >> ***** Running training *****
[INFO|trainer.py:2403] 2025-07-11 07:25:21,245 >>   Num examples = 35,522
[INFO|trainer.py:2404] 2025-07-11 07:25:21,246 >>   Num Epochs = 1
[INFO|trainer.py:2405] 2025-07-11 07:25:21,247 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:2408] 2025-07-11 07:25:21,248 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer.py:2409] 2025-07-11 07:25:21,248 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2410] 2025-07-11 07:25:21,249 >>   Total optimization steps = 2,221
[INFO|trainer.py:2411] 2025-07-11 07:25:21,250 >>   Number of trainable parameters = 135,328,517


Step,Training Loss
500,1.083500
1000,0.966200
1500,0.912700
2000,0.883300


[INFO|trainer.py:2677] 2025-07-11 07:32:53,041 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)




#### Saving the tokenizer too for easy upload

In [34]:
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|trainer.py:4019] 2025-07-11 07:32:53,096 >> Saving model checkpoint to ./distilBERT_m/
[INFO|configuration_utils.py:437] 2025-07-11 07:32:53,099 >> Configuration saved in ./distilBERT_m/config.json
[INFO|modeling_utils.py:3949] 2025-07-11 07:33:01,067 >> Model weights saved in ./distilBERT_m/model.safetensors
[INFO|tokenization_utils_base.py:2507] 2025-07-11 07:33:01,072 >> tokenizer config file saved in ./distilBERT_m/tokenizer_config.json
[INFO|tokenization_utils_base.py:2516] 2025-07-11 07:33:01,073 >> Special tokens file saved in ./distilBERT_m/special_tokens_map.json


***** train metrics *****
  epoch                    =        1.0
  total_flos               =  1095644GF
  train_loss               =     0.9517
  train_runtime            = 0:07:31.83
  train_samples            =      35522
  train_samples_per_second =     78.618
  train_steps_per_second   =      4.916


#### Evaluating our model on validation/development data

In [35]:
logger.info("*** Evaluate ***")

metrics = trainer.evaluate(eval_dataset=eval_dataset)

max_eval_samples = (
    max_eval_samples if max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

INFO:__main__:*** Evaluate ***
[INFO|trainer.py:4353] 2025-07-11 07:33:01,178 >> 
***** Running Evaluation *****
[INFO|trainer.py:4355] 2025-07-11 07:33:01,178 >>   Num examples = 2512
[INFO|trainer.py:4358] 2025-07-11 07:33:01,179 >>   Batch size = 16


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.6939
  eval_loss               =     0.8355
  eval_runtime            = 0:00:08.92
  eval_samples            =       2512
  eval_samples_per_second =    281.499
  eval_steps_per_second   =     17.594


### Predecting the test data

In [36]:
id2l = {v: k for k, v in l2id.items()}
logger.info("*** Predict ***")
#predict_dataset = predict_dataset.remove_columns("label")
ids = predict_dataset['id']
predict_dataset = predict_dataset.remove_columns("id")
predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)
output_predict_file = os.path.join(training_args.output_dir, f"subtask_1B.tsv")
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as writer:
        logger.info(f"***** Predict results *****")
        writer.write("id\tlabel\tmodel\n")
        for index, item in enumerate(predictions):
            item = label_list[item]
            item = id2l[item]
            writer.write(f"{ids[index]}\t{item}\t{model_name}\n")

INFO:__main__:*** Predict ***
[INFO|trainer.py:4353] 2025-07-11 07:33:10,551 >> 
***** Running Prediction *****
[INFO|trainer.py:4355] 2025-07-11 07:33:10,551 >>   Num examples = 2512
[INFO|trainer.py:4358] 2025-07-11 07:33:10,552 >>   Batch size = 16


INFO:__main__:***** Predict results *****


In [37]:
ids[0]

879187

#### Saving the model into card

In [38]:
kwargs = {"finetuned_from": model_name, "tasks": "text-classification"}
trainer.create_model_card(**kwargs)

[INFO|modelcard.py:450] 2025-07-11 07:33:19,898 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.693869411945343}]}


In [39]:
!zip subtask_1B.zip ./distilBERT_m/subtask_1B.tsv

  adding: distilBERT_m/subtask_1B.tsv (deflated 90%)
